<a href="https://colab.research.google.com/github/aguspatur22/SMS-Text-Classification/blob/main/Spam_or_Ham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.utils import resample
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.0/590.0 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.9/584.9 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 24.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Fou

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-04-05 22:19:15--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2024-04-05 22:19:15 (10.5 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-04-05 22:19:15--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [ ]:
# Load data and set column names
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Convert labels to binary format
train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

# Tokenize the messages
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_data['message'])

train_data['message'] = tokenizer.texts_to_sequences(train_data['message'])
test_data['message'] = tokenizer.texts_to_sequences(test_data['message'])

# Pad the sequences so they're all the same length
train_data['message'] = pad_sequences(train_data['message'], padding='post')
test_data['message'] = pad_sequences(test_data['message'], padding='post')

# Separate majority and minority classes
data_majority = train_data[train_data.label==0]
data_minority = train_data[train_data.label==1]

# Upsample minority class
data_minority_upsampled = resample(data_minority,
                                 replace=True,     # sample with replacement
                                 n_samples=data_majority.shape[0],    # to match majority class
                                 random_state=123) # reproducible results

# Combine majority class with upsampled minority class
train_data_balanced = pd.concat([data_majority, data_minority_upsampled])

# Build the model
model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=10000, output_dim=32),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.Dropout(0.2),
    keras.layers.LSTM(32),
    keras.layers.Dropout(0.2),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model with RMSprop optimizer
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.003), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with the balanced data for more epochs
model.fit(train_data_balanced['message'], train_data_balanced['label'], epochs=16)

Epoch 1/16
227/227 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7669 - loss: 0.4609
Epoch 2/16
227/227 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9084 - loss: 0.2567
Epoch 3/16
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9057 - loss: 0.2522
Epoch 4/16
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9165 - loss: 0.2343
Epoch 5/16
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9098 - loss: 0.2423
Epoch 6/16
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9072 - loss: 0.2407
Epoch 7/16
227/227 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9164 - loss: 0.2247
Epoch 8/16
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9131 - loss: 0.2329
Epoch 9/16
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9160 - loss: 0.2310
Epoch 10/16
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9127 - loss: 0.2287
Epoch 11/16
227/227 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9165 - loss: 0.2209
Epoch 12/16
227/227 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/ste

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # Preprocess the message
    message = tokenizer.texts_to_sequences([pred_text])
    message = pad_sequences(message, padding='post')

    # Use the model to predict the label
    prediction = model.predict(message)

    # The prediction is a number between 0 and 1, classify it as 'ham' if it's less than 0.5 and 'spam' otherwise
    return [prediction[0][0], "ham" if prediction[0][0] < 0.4 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[0.0034961547, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[0.0034961547, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[0.5083574, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[0.36065853, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[0.44582048, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[0.871931, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[9.15812e-06, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[2.9313298e-05, 'ham']
You passed the challenge. Great job!
